<a href="https://colab.research.google.com/github/ranadeepbhuyan/ml-class/blob/master/3d_to_2d_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall nibabel
!pip install nibabel==5.1.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os, glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import cv2

In [ ]:
import gzip
import os
import shutil

# set the root directory
root_dir = "/content/drive/MyDrive/training_dataset_nii/nii 50-100"

# recursively traverse the directory tree
for subdir, dirs, files in os.walk(root_dir):

    # loop through each file in the directory
    for file_name in files:

        # check if the file is a gzipped file
        if file_name.endswith(".gz"):

            # get the full path of the file
            file_path = os.path.join(subdir, file_name)

            # set the output file name
            output_file_name = os.path.splitext(file_path)[0]

            # open the gzipped file for reading
            with gzip.open(file_path, 'rb') as f_in:

                # open the output file for writing
                with open(output_file_name, 'wb') as f_out:

                    # copy the contents of the gzipped file to the output file
                    shutil.copyfileobj(f_in, f_out)

            # delete the gzipped file after extraction (optional)
            os.remove(file_path)


In [ ]:
TRAIN_DATASET_PATH = r"/content/drive/MyDrive/training_dataset_nii/nii 50-100" +'/'

In [ ]:
# STEP 2 - Image normalization
HOUNSFIELD_MIN = 0.0
HOUNSFIELD_MAX = 3500
HOUNSFIELD_RANGE = HOUNSFIELD_MAX - HOUNSFIELD_MIN

# STEP 3 - Slicing and saving
SLICE_X = True
SLICE_Y = True
SLICE_Z = False

SLICE_DECIMATE_IDENTIFIER = 3

In [ ]:
# Normalize image
def normalizeImageIntensityRange(img):
    img[img < HOUNSFIELD_MIN] = HOUNSFIELD_MIN
    img[img > HOUNSFIELD_MAX] = HOUNSFIELD_MAX
    return (img - HOUNSFIELD_MIN) / HOUNSFIELD_RANGE

#nImg = normalizeImageIntensityRange(img)
#np.min(nImg), np.max(nImg), nImg.shape, type(nImg)

In [ ]:
# lists of directories with studies
train_and_val_directories = [f.path for f in os.scandir(TRAIN_DATASET_PATH) if f.is_dir()]

def pathListIntoIds(dirList):
    x = []
    for i in range(0,len(dirList)):
        x.append(dirList[i][dirList[i].rfind('/')+1:])
    return x

train_and_test_ids = pathListIntoIds(train_and_val_directories);

In [ ]:
def remove_folders(dataset):
  for i in range(0,len(dataset)):
    test = dataset[i].split('_nifti')[0].replace('-',' ')[0]
    a = 'U'
    if test != a:
      dataset.pop(i)
  return dataset


In [ ]:
remove_folders(train_and_test_ids)





In [ ]:
len(train_and_test_ids)

87

In [ ]:
train_and_test_ids.sort()

In [ ]:
train_and_test_ids

In [ ]:
def remove_nifi(dataset):
  test2 = []
  for i in range(0,len(train_and_test_ids)):
    test = train_and_test_ids[i].split('_nifti')[0]
    test2.append(test)
  return test2


In [ ]:
training_ids = remove_nifi(train_and_test_ids)

In [ ]:
count = 0
for c, i in enumerate(training_ids):
      case_path = os.path.join(TRAIN_DATASET_PATH, f'{i}_nifti')

      data_path = os.path.join(case_path, f'{i}_FLAIR.nii');
      flair = nib.load(data_path).get_fdata()

      data_path = os.path.join(case_path, f'{i}_T1c.nii');
      ce = nib.load(data_path).get_fdata()

      data_path = os.path.join(case_path, f'{i}_T2.nii');
      t2 = nib.load(data_path).get_fdata()

      data_path = os.path.join(case_path, f'{i}_tumor_segmentation.nii');
      seg = nib.load(data_path).get_fdata()

In [ ]:
traindata = flair + ce + t2

In [ ]:
seg.shape

(240, 240, 155)

In [ ]:
traindata.shape

(240, 240, 155)

In [ ]:
imageSliceOutput = r"/content/drive/MyDrive/training_dataset_nii/3d_to_2d_dataset"

maskSliceOutput = r"/content/drive/MyDrive/training_dataset_nii/MASK1"

In [ ]:
# Read image or mask volume
def readImageVolume(img, normalize=False):
    if normalize:
        return normalizeImageIntensityRange(img)
    else:
        return img
readImageVolume(traindata, normalize=True)
readImageVolume(seg, normalize=False)

memmap([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
    

In [ ]:
#test_path = r"/content/drive/MyDrive/dataset _nii_files/test_image"

In [ ]:
def saveSlice(img, fname, path):
    img = np.uint8(img * 255)
    fout = os.path.join(path, f'{fname}.png')
    cv2.imwrite(fout, img)
    print(f'[+] Slice saved: {fout}', end='\r')

#saveSlice(traindata[125,:,:], 'test',test_path )
#saveSlice(seg[:,105,:] , 'test3', test_path)

In [ ]:
# Slice image in all directions and save
def sliceAndSaveVolumeImage(vol, fname, path):
    (dimx, dimy, dimz) = vol.shape
    print(dimx, dimy, dimz)
    cnt = 0
    if SLICE_X:
        cnt += dimx
        print('Slicing X: ')
        for i in range(dimx):
            saveSlice(vol[i,:,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x', path)

    if SLICE_Y:
        cnt += dimy
        print('Slicing Y: ')
        for i in range(dimy):
            saveSlice(vol[:,i,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y', path)

    if SLICE_Z:
        cnt += dimz
        print('Slicing Z: ')
        for i in range(dimz):
            saveSlice(vol[:,:,i], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', path)
    return cnt

In [ ]:
# Read and process image volumes
for filename in train_and_test_ids:
    img = readImageVolume(traindata, True)
    print(filename, img.shape, np.sum(img.shape), np.min(img), np.max(img))
    numOfSlices = sliceAndSaveVolumeImage(img, filename, imageSliceOutput)
    print(f'\n{filename}, {numOfSlices} slices created \n')

UCSF-PDGM-0004_nifti (240, 240, 155) 635 0.0 1.0
240 240 155
Slicing X: 
Slicing Y: 
[+] Slice saved: /content/drive/MyDrive/training_dataset_nii/3d_to_2d_dataset/UCSF-PDGM-0004_nifti-slice239_y.png
UCSF-PDGM-0004_nifti, 480 slices created 

UCSF-PDGM-0005_nifti (240, 240, 155) 635 0.0 1.0
240 240 155
Slicing X: 
Slicing Y: 
[+] Slice saved: /content/drive/MyDrive/training_dataset_nii/3d_to_2d_dataset/UCSF-PDGM-0005_nifti-slice239_y.png
UCSF-PDGM-0005_nifti, 480 slices created 

UCSF-PDGM-0007_nifti (240, 240, 155) 635 0.0 1.0
240 240 155
Slicing X: 
Slicing Y: 
[+] Slice saved: /content/drive/MyDrive/training_dataset_nii/3d_to_2d_dataset/UCSF-PDGM-0007_nifti-slice239_y.png
UCSF-PDGM-0007_nifti, 480 slices created 

UCSF-PDGM-0008_nifti (240, 240, 155) 635 0.0 1.0
240 240 155
Slicing X: 
Slicing Y: 
[+] Slice saved: /content/drive/MyDrive/training_dataset_nii/3d_to_2d_dataset/UCSF-PDGM-0008_nifti-slice239_y.png
UCSF-PDGM-0008_nifti, 480 slices created 

UCSF-PDGM-0009_nifti (240, 240, 

In [ ]:
# Read and process image mask volumes
for filename in train_and_test_ids:
  img = readImageVolume(seg, False)
  print(filename, img.shape, np.sum(img.shape), np.min(img), np.max(img))
  numOfSlices = sliceAndSaveVolumeImage(img, filename, maskSliceOutput)
  print(f'\n{filename}, {numOfSlices} slices created \n')

UCSF-PDGM-0004_nifti (240, 240, 155) 635 0.0 4.0
240 240 155
Slicing X: 
Slicing Y: 
[+] Slice saved: /content/drive/MyDrive/training_dataset_nii/MASK1/UCSF-PDGM-0004_nifti-slice239_y.png
UCSF-PDGM-0004_nifti, 480 slices created 

UCSF-PDGM-0005_nifti (240, 240, 155) 635 0.0 4.0
240 240 155
Slicing X: 
Slicing Y: 
[+] Slice saved: /content/drive/MyDrive/training_dataset_nii/MASK1/UCSF-PDGM-0005_nifti-slice239_y.png
UCSF-PDGM-0005_nifti, 480 slices created 

UCSF-PDGM-0007_nifti (240, 240, 155) 635 0.0 4.0
240 240 155
Slicing X: 
Slicing Y: 
[+] Slice saved: /content/drive/MyDrive/training_dataset_nii/MASK1/UCSF-PDGM-0007_nifti-slice239_y.png
UCSF-PDGM-0007_nifti, 480 slices created 

UCSF-PDGM-0008_nifti (240, 240, 155) 635 0.0 4.0
240 240 155
Slicing X: 
Slicing Y: 
[+] Slice saved: /content/drive/MyDrive/training_dataset_nii/MASK1/UCSF-PDGM-0008_nifti-slice239_y.png
UCSF-PDGM-0008_nifti, 480 slices created 

UCSF-PDGM-0009_nifti (240, 240, 155) 635 0.0 4.0
240 240 155
Slicing X: 
Sli